In [36]:
#import the libraries

from elasticsearch import Elasticsearch
import json
import re
import pandas as pd
import numpy as np
import bs4
import lxml 
import os
import string
from bs4 import BeautifulSoup

In [2]:
#initializing the instance of elasticsearch to localhost at port 9200(default)
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [3]:
copyright = open("dictionaries/_copyright.dict","r")
x = copyright.readlines()
_copyright = dict()
for i in range(len(x)):
    _copyright[i+1] = x[i].strip()

In [4]:
court = open("dictionaries/_court.dict","r")
x = court.readlines()

_court = dict()

for i in range(len(x)):
    _court[i+1] = x[i].strip()

In [5]:
cprocedure = open("dictionaries/_criminalprocedure.dict","r")
x = cprocedure.readlines()

_criminalprocedure = dict()

for i in range(len(x)):
    _criminalprocedure[i+1] = x[i].strip()

In [6]:
design = open("dictionaries/_design.dict","r")
x = design.readlines()

_design = dict()

for i in range(len(x)):
    _design[i+1] = x[i].strip()

In [7]:
entity = open("dictionaries/_entity.dict","r")
x = entity.readlines()

_entity = dict()

for i in range(len(x)):
    _entity[i+1] = x[i].strip()

In [8]:
legalgen = open("dictionaries/_legalgen.dict","r")
x = legalgen.readlines()

_legalgen = dict()

for i in range(len(x)):
    _legalgen[i+1] = x[i].strip()

In [9]:
patent = open("dictionaries/_patent.dict","r")
x = patent.readlines()

_patent = dict()

for i in range(len(x)):
    _patent[i+1] = x[i].strip()

In [10]:
reservation = open("dictionaries/_reservation.dict","r")
x = reservation.readlines()

_reservation = dict()

for i in range(len(x)):
    _reservation[i+1] = x[i].strip()

In [11]:
scjudge = open("dictionaries/_scjudge.dict","r")
x = scjudge.readlines()

_scjudge = dict()

for i in range(len(x)):
    _scjudge[i+1] = x[i].strip()

In [12]:
trademark = open("dictionaries/_trademark.dict","r")
x = trademark.readlines()

_trademark = dict()

for i in range(len(x)):
    _trademark[i+1] = x[i].strip()

In [13]:
#load the json file

file = open("tag.json","r",encoding="utf-8")

data = json.load(file)
file.close()

In [14]:
tids = list(data.keys())

In [55]:
for i in tids: 
    es.indices.delete(index=i, ignore=[400, 404])

C:\Users\Jayant\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [56]:
#create the index

for i in tids[902:]: 
    es.indices.delete(index=i, ignore=[400, 404])
    es.indices.create(index=i)



In [57]:
#insert the data in the index

for i in tids[902:]:
    es.index(index=i, doc_type="cases", id=int(i), body=data[i])

C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  after removing the cwd from sys.path.
C:\Users\Jayant\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


In [58]:
def get_count(res):
    if len(res['hits']['hits']) == 0:
        return 0
    else:
        #check if phrase freq
        freq = res['hits']['hits'][0]['_explanation']['details'][0]['details'][2]['details'][0]['description']
        rg = re.compile(r'[+-]?([0-9]*[.])?[0-9]+')
        result = rg.search(freq)
        if result:
            return int(float(result.group()))
        else:
            freq = res['hits']['hits'][0]['_explanation']['details'][0]['description']
            result = rg.search(freq)
            if result:
                return int(float(result.group()))
            else:
                return 0

In [59]:
#rough pseudocode

def return_count_per_dict(tid , dictionary , desc):
    term_dict = {}
    where_to_search = ""
    if str(desc) == "judge":
        where_to_search = "author"
    else:
        where_to_search = "content"
        
    for i in dictionary.keys():
        term_dict[dictionary[i]] = 0
    for i in dictionary.keys():
        term = dictionary[i]
        #search for the term
        res = es.search(index=tid, body={"size":1000,"query": {"match_phrase": {where_to_search: str(term)}}, "explain":True})
        #get count
        count = get_count(res)
        term_dict[dictionary[i]] = count
    
    #get max count
    max_term = max(zip(term_dict.values(), term_dict.keys()))[1]
    if term_dict[max_term] == 0:
        return 0
    else:
        for key, value in dictionary.items():
            if max_term == value:
                return int(key)

In [21]:
tfiles = []
os.chdir("html_files")
for file in os.listdir():
    if file.endswith(".html"):
        tfiles.append(int(file[:-5]))

In [22]:
os.chdir("..")

In [60]:
feature = pd.read_csv("updated_oe.csv")

In [61]:
feature

,tid,LEGAL_SCJUDGE,LEGAL_DESIGN,LEGAL_GEN,LEGAL_PATENT,LEGAL_TRADEMARK,LEGAL_COPYRIGHT,LEGAL_RESERVATION_CATEGORY,LEGAL_CRIMINAL_PROCEDURE,LEGAL_ENTITY,COPYRIGHT_ACT,PATENTS_ACT,DESIGNS_ACT,INFORMATION_TECHNOLOGY_ACT,TRADEMARK_ACT
0,100123,0,6,17,0,0,0,0,209,10,0,0,0,0,0
1,1001948,0,4,31,2,0,5,4,66,10,1,0,0,0,0
2,1006468,0,4,31,0,3,0,0,191,17,0,0,0,0,0
3,1006709,0,0,22,0,0,0,0,66,1,0,0,0,0,0
4,1007946,0,0,17,0,0,3,0,66,17,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,99622088,0,0,0,0,0,0,0,0,0,0,0,0,0,0
954,996328,0,0,0,0,0,0,0,0,0,0,0,0,0,0
955,996348,0,0,0,0,0,0,0,0,0,0,0,0,0,0
956,99670896,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
feature.iloc[801]

tid                           712132
LEGAL_SCJUDGE                      0
LEGAL_DESIGN                       4
LEGAL_GEN                          5
LEGAL_PATENT                       0
LEGAL_TRADEMARK                   62
LEGAL_COPYRIGHT                    5
LEGAL_RESERVATION_CATEGORY         0
LEGAL_CRIMINAL_PROCEDURE         191
LEGAL_ENTITY                      17
COPYRIGHT_ACT                      1
PATENTS_ACT                        0
DESIGNS_ACT                        0
INFORMATION_TECHNOLOGY_ACT         0
TRADEMARK_ACT                      0
Name: 801, dtype: int64

In [62]:
dicts = { "judge" : _scjudge , "design" : _design , "legalgen" : _legalgen , "patent" : _patent , "trademark" : _trademark ,  "copyright" : _copyright , "reservation" : _reservation , "crpc" : _criminalprocedure , "entity" : _entity} 

In [63]:
for i in range(902,feature.shape[0]):
    tid = feature.iloc[i,0]
    #pick dictionary one by one
    count = 0
    for j in dicts.keys():
        dictionary = dicts[j]
        feature.iloc[i,count+1] = return_count_per_dict(tid , dictionary , j)
        count = count + 1

C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  app.launch_new_instance()
C:\Users\Jayant\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [64]:
acts = ['copyright act' , 'patents act' , 'designs act' , 'information technology act' , 'trademark act']

In [65]:
def get_count_of_act(tid , act):
    res = es.search(index=tid, body={"size":1000,"query": {"match_phrase": {"content": str(act)}}, "explain":True})
    count = get_count(res)
    if count == 0:
        return 0
    else:
        return 1

In [66]:
for i in range(902,feature.shape[0]):
    tid = feature.iloc[i,0]
    
    feature.iloc[i,10] = get_count_of_act(tid , acts[0])
    feature.iloc[i,11] = get_count_of_act(tid , acts[1])
    feature.iloc[i,12] = get_count_of_act(tid , acts[2])
    feature.iloc[i,13] = get_count_of_act(tid , acts[3])
    feature.iloc[i,14] = get_count_of_act(tid , acts[4])
        

C:\Users\Jayant\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


In [67]:
feature

,tid,LEGAL_SCJUDGE,LEGAL_DESIGN,LEGAL_GEN,LEGAL_PATENT,LEGAL_TRADEMARK,LEGAL_COPYRIGHT,LEGAL_RESERVATION_CATEGORY,LEGAL_CRIMINAL_PROCEDURE,LEGAL_ENTITY,COPYRIGHT_ACT,PATENTS_ACT,DESIGNS_ACT,INFORMATION_TECHNOLOGY_ACT,TRADEMARK_ACT
0,100123,0,6,17,0,0,0,0,209,10,0,0,0,0,0
1,1001948,0,4,31,2,0,5,4,66,10,1,0,0,0,0
2,1006468,0,4,31,0,3,0,0,191,17,0,0,0,0,0
3,1006709,0,0,22,0,0,0,0,66,1,0,0,0,0,0
4,1007946,0,0,17,0,0,3,0,66,17,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,99622088,0,7,31,0,62,5,4,191,10,1,0,0,1,0
954,996328,0,8,31,0,0,3,0,191,17,0,0,0,0,0
955,996348,0,6,31,0,0,0,5,191,21,0,0,0,0,0
956,99670896,0,4,31,0,62,5,5,191,10,0,0,1,0,0


In [40]:
feature.iloc[602]

tid                           357132
LEGAL_SCJUDGE                      0
LEGAL_DESIGN                       0
LEGAL_GEN                         31
LEGAL_PATENT                       0
LEGAL_TRADEMARK                    0
LEGAL_COPYRIGHT                    0
LEGAL_RESERVATION_CATEGORY         0
LEGAL_CRIMINAL_PROCEDURE          66
LEGAL_ENTITY                       3
COPYRIGHT_ACT                      0
PATENTS_ACT                        0
DESIGNS_ACT                        0
INFORMATION_TECHNOLOGY_ACT         0
TRADEMARK_ACT                      0
Name: 602, dtype: int64

In [68]:
feature.to_csv("updated_oe.csv",index=False)

In [69]:
#merge old and new updated features

In [70]:
old = pd.read_csv("updated_oe.csv")

In [71]:
new = pd.read_csv("features_2020.csv")

In [72]:
old.shape[0],new.shape[0]

(958, 96)

In [73]:
new.columns

Index(['tid', 'LEGAL_SCJUDGE', 'LEGAL_DESIGN', 'LEGAL_GEN', 'LEGAL_PATENT',
       'LEGAL_TRADEMARK', 'LEGAL_COPYRIGHT', 'LEGAL_RESERVATION_CATEGORY',
       'LEGAL_CRIMINAL_PROCEDURE', 'LEGAL_ENTITY', 'LAW', 'INFRINGEMENT',
       'PERSON', 'NATION/RELIGIOUS/POL_GROUP', 'GEOGRAPHICAL_LOC', 'LANGUAGE',
       'LOCATION_STRUCTURE', 'LOCATION', 'CONTACTNAME', 'ORGANIZATION',
       'ORGANIZATION2', 'GEOGRAPHY2', 'DATETIME', 'PERCENT', 'COPYRIGHT_ACT',
       'PATENTS_ACT', 'DESIGNS_ACT', 'INFORMATION_TECHNOLOGY_ACT',
       'TRADEMARK_ACT'],
      dtype='object')

In [74]:
new.drop(['LAW', 'INFRINGEMENT',
       'PERSON', 'NATION/RELIGIOUS/POL_GROUP', 'GEOGRAPHICAL_LOC', 'LANGUAGE',
       'LOCATION_STRUCTURE', 'LOCATION', 'CONTACTNAME', 'ORGANIZATION',
       'ORGANIZATION2', 'GEOGRAPHY2', 'DATETIME', 'PERCENT'],axis=1,inplace=True)

In [75]:
new

,tid,LEGAL_SCJUDGE,LEGAL_DESIGN,LEGAL_GEN,LEGAL_PATENT,LEGAL_TRADEMARK,LEGAL_COPYRIGHT,LEGAL_RESERVATION_CATEGORY,LEGAL_CRIMINAL_PROCEDURE,LEGAL_ENTITY,COPYRIGHT_ACT,PATENTS_ACT,DESIGNS_ACT,INFORMATION_TECHNOLOGY_ACT,TRADEMARK_ACT
0,1004544,0,7,24,0,62,0,0,209,10,0,0,0,0,0
1,102042281,0,4,5,0,62,3,4,78,8,0,0,0,0,0
2,1025815,0,4,27,3,62,0,4,209,10,0,0,0,0,0
3,1062207,0,4,31,0,62,22,0,209,10,0,0,0,0,0
4,106681665,0,0,31,0,0,0,0,54,6,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,8433581,0,8,2,0,62,5,4,209,10,1,0,0,0,0
92,850381,0,4,31,3,62,0,5,191,17,0,0,0,0,0
93,858066,0,4,31,0,62,3,4,191,10,1,0,0,0,0
94,94612663,0,4,31,0,62,0,4,66,17,0,0,0,0,1


In [76]:
new.to_csv("features_2020.csv",index=False)

In [77]:
df = pd.concat(
    map(pd.read_csv, ['updated_oe.csv', 'features_2020.csv']), ignore_index=True)

In [78]:
df.shape[0]

1054

In [81]:
df[df.duplicated()]

,tid,LEGAL_SCJUDGE,LEGAL_DESIGN,LEGAL_GEN,LEGAL_PATENT,LEGAL_TRADEMARK,LEGAL_COPYRIGHT,LEGAL_RESERVATION_CATEGORY,LEGAL_CRIMINAL_PROCEDURE,LEGAL_ENTITY,COPYRIGHT_ACT,PATENTS_ACT,DESIGNS_ACT,INFORMATION_TECHNOLOGY_ACT,TRADEMARK_ACT
963,1114158,0,4,27,0,6,0,4,191,17,0,0,0,0,0
970,1239673,0,4,31,0,62,0,0,66,10,0,0,0,0,0
972,1281050,0,7,11,0,0,3,5,191,17,0,0,0,0,0
993,1674695,0,4,31,0,62,5,4,191,10,0,0,0,0,0
995,172383107,0,4,31,0,0,0,4,66,1,0,0,0,0,0
997,1771880,0,4,31,0,38,0,4,191,17,0,0,0,0,0
1007,19267666,0,4,31,0,62,5,4,66,10,0,1,1,0,0
1017,330608,0,7,22,0,0,3,0,66,1,0,0,0,0,0
1021,421030,0,7,31,0,0,0,4,66,17,0,0,0,0,0
1024,508915,0,4,27,3,0,0,4,191,10,0,0,0,0,0


In [83]:
df.drop_duplicates(inplace=True)

In [84]:
df.shape[0]

1038

In [85]:
df.to_csv("FINAL_OE_LABELS.csv",index=False)